In [ ]:
import glofrim.gbmi_lib as glib 
from configparser import ConfigParser
from os.path import join, isfile, abspath, dirname, basename, normpath

In [ ]:
config_fn = r'/home/dirk/repos/model_test_data/test_Elbe/glofrim.ini'
env_fn = r'/home/dirk/repos/glofrim/environment.ini'

In [ ]:
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)
env.sections()

In [ ]:
from glofrim import Glofrim
cbmi = Glofrim()
cbmi.initialize_config(config_fn, env_fn=env_fn)


In [ ]:
cbmi.initialize_model()

In [ ]:
from pprint import pprint
sc = cbmi.exchanges
pprint(sc)

In [ ]:
cbmi.get_start_time(), cbmi.get_attribute_value('CMF.SIMTIME:ISYEAR'), cbmi.get_attribute_value('CMF.SIMTIME:ISMON'), 

In [ ]:
cbmi.bmimodels['PCR'].get_start_time(), cbmi.bmimodels['CMF'].get_start_time()

In [ ]:
from datetime import datetime
cbmi.update_until(datetime(2000, 1, 10))
cbmi.bmimodels['CMF']._t

In [ ]:
cf = cbmi._config
cf.has_option('coupling', 'startTime')